# Import Honeypot Data

Downloaded the honeypot data from https://www.secrepo.com/honeypot/honeypot.json.zip  

Now need to figure out what sort of work is required to make it usable.

In [6]:
import json
import pandas as pd
import sys

Can I even straight-up load with pandas? 

Answer was no -- needs custom loading function

In [2]:
with open('../data/honeypot.json') as f:
    jsons = [json.loads(l) for l in f.readlines()]
#    dfs = [pd.DataFrame(j) for j in jsons]

In [17]:
def build_df_row(j):
    try:
        if j['channel'] == 'glastopf.events':
            payload = json.loads(j['payload'])
            row = pd.DataFrame({
                'id'             : j['_id']['$oid'],
                'ident'          : j['ident'],
                'normalized'     : j['normalized'],
                'timestamp'      : j['timestamp']['$date'],
                'channel'        : j['channel'],
                'pattern'        : payload['pattern'],
                'filename'       : payload['filename'],
                'request_raw'    : payload['request_raw'],
                'request_url'    : payload['request_url'],
                'attackerIP'     : payload['source'][0],
                'attackerPort'   : payload['source'][1],
                'victimPort'     : 80, # from documentation,
                'victimIP'       : 0 # might need to do something else with this, see documentation
            }, index = [0])
        elif j['channel'] == 'amun.events':
            payload = json.loads(j['payload'])
            row = pd.DataFrame({
                'id'             : j['_id']['$oid'],
                'ident'          : j['ident'],
                'normalized'     : j['normalized'],
                'timestamp'      : j['timestamp']['$date'],
                'channel'        : j['channel'],
                'attackerIP'     : payload['attackerIP'],
                'attackerPort'   : payload['attackerPort'],
                'victimIP'       : payload['victimIP'],
                'victimPort'     : payload['victimPort'],
                'connectionType' : payload['connectionType']      
            }, index = [0])
        elif j['channel'] == 'dionaea.connections':
            payload = json.loads(j['payload'])
            row = pd.DataFrame({
                'id'                  : j['_id']['$oid'],
                'ident'               : j['ident'],
                'normalized'          : j['normalized'],
                'timestamp'           : j['timestamp']['$date'],
                'channel'             : j['channel'],
                'attackerIP'          : payload['remote_host'],
                'attackerPort'        : payload['remote_port'],
                'victimIP'            : payload['local_host'],
                'victimPort'          : payload['local_port'],
                'connectionType'      : payload['connection_type'],
                'connectionTransport' : payload['connection_transport'],
                'connectionProtocol'  : payload['connection_protocol'],
                'remoteHostname'      : payload['remote_hostname']
            }, index = [0])
        elif j['channel'] == 'snort.alerts':
            payload = json.loads(j['payload'])
            row = pd.DataFrame({
                'id'                  : j['_id']['$oid'],
                'ident'               : j['ident'],
                'normalized'          : j['normalized'],
                'timestamp'           : j['timestamp']['$date'],
                'channel'             : j['channel'],
                'attackerIP'          : payload['source_ip'],
                'victimIP'            : payload['destination_ip'],
                'connectionType'      : payload['classification'],
                'connectionProtocol'  : payload['proto'],
                'priority'            : payload['priority'],
                'header'              : payload['header'],
                'signature'           : payload['signature'],
                'sensor'              : payload['sensor']
            }, index = [0])
        else:
            row = pd.DataFrame()
        return row
    except:
        print(sys.exc_info())
        print(j)

{'_id': {'$oid': '542820019f8c6d41306aeaff'}, 'ident': '139cfdf2-471e-11e4-9ee4-0a0b6e7c3e9e', 'timestamp': {'$date': '2014-09-28T14:49:37.787+0000'}, 'normalized': True, 'payload': '{"destination_ip": "172.31.13.124", "classification": "Misc activity", "proto": "ICMP", "source_ip": "201.158.32.1", "priority": "3", "header": "1:486:4", "signature": "ICMP Destination Unreachable Communication with Destination Host is Administratively Prohibited ", "date": "2014-09-28T14:49:43.391463", "sensor": "139cfdf2-471e-11e4-9ee4-0a0b6e7c3e9e"}', 'channel': 'snort.alerts'}
{'_id': {'$oid': '54283a699f8c6d41306aeb31'}, 'ident': 'e93b34b2-4726-11e4-9ee4-0a0b6e7c3e9e', 'timestamp': {'$date': '2014-09-28T16:42:17.698+0000'}, 'normalized': True, 'payload': '{"destination_ip": "172.31.14.66", "classification": "Misc activity", "proto": "ICMP", "source_ip": "218.77.79.58", "priority": "3", "header": "1:486:4", "signature": "ICMP Destination Unreachable Communication with Destination Host is Administratively Prohibited ", "date": "2014-09-28T16:42:31.769696", "sensor": "e93b34b2-4726-11e4-9ee4-0a0b6e7c3e9e"}', 'channel': 'snort.alerts'}

In [18]:
#tj = {'_id': {'$oid': '542820019f8c6d41306aeaff'}, 'ident': '139cfdf2-471e-11e4-9ee4-0a0b6e7c3e9e', 'timestamp': {'$date': '2014-09-28T14:49:37.787+0000'}, 'normalized': True, 'payload': '{"destination_ip": "172.31.13.124", "classification": "Misc activity", "proto": "ICMP", "source_ip": "201.158.32.1", "priority": "3", "header": "1:486:4", "signature": "ICMP Destination Unreachable Communication with Destination Host is Administratively Prohibited ", "date": "2014-09-28T14:49:43.391463", "sensor": "139cfdf2-471e-11e4-9ee4-0a0b6e7c3e9e"}', 'channel': 'snort.alerts'}

In [19]:
#build_df_row(tj)

In [24]:
# df = pd.concat([build_df_row(j) for j in jsons], sort=False, )
# df.reset_index(inplace=True, drop=True)
df.head()

,id,ident,normalized,timestamp,channel,pattern,filename,request_raw,request_url,attackerIP,...,victimPort,victimIP,connectionType,connectionProtocol,priority,header,signature,sensor,connectionTransport,remoteHostname
0,5426456e9f8c6d41306aea57,a16f5f36-3c41-11e4-9ee4-0a0b6e7c3e9e,True,2014-09-27T05:04:46.363+0000,glastopf.events,style_css,None,"GET /style.css HTTP/1.1\r\nAccept: text/css,*/...",/style.css,162.197.24.67,...,80.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,542645799f8c6d41306aea59,a16f5f36-3c41-11e4-9ee4-0a0b6e7c3e9e,True,2014-09-27T05:04:57.901+0000,glastopf.events,unknown,None,"POST /index HTTP/1.1\r\nAccept: text/html,appl...",/index,162.197.24.67,...,80.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5426457a9f8c6d41306aea5a,a16f5f36-3c41-11e4-9ee4-0a0b6e7c3e9e,True,2014-09-27T05:04:58.066+0000,glastopf.events,style_css,None,"GET /style.css HTTP/1.1\r\nAccept: text/css,*/...",/style.css,162.197.24.67,...,80.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5426457a9f8c6d41306aea5d,a16f5f36-3c41-11e4-9ee4-0a0b6e7c3e9e,True,2014-09-27T05:04:58.248+0000,glastopf.events,unknown,None,GET /favicon.ico HTTP/1.1\r\nAccept: */*\r\nAc...,/favicon.ico,162.197.24.67,...,80.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5426462d9f8c6d41306aea5f,a16f5f36-3c41-11e4-9ee4-0a0b6e7c3e9e,True,2014-09-27T05:07:57.267+0000,glastopf.events,unknown,None,"GET /index HTTP/1.1\r\nAccept: text/html,appli...",/index,173.174.71.56,...,80.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
# Let's save the df off so we don't have to spend so long re-creating it if needed.
#df.to_pickle('../data/honeypot_df.pkl')

In [21]:
df.shape

(994142, 21)

In [22]:
df['connectionType'].value_counts()

reject                         437210
initial                        366765
connect                        114355
Potentially Bad Traffic         30953
accept                          23513
Misc activity                    9483
exploit                          4552
listen                            102
Attempted Denial of Service        10
Name: connectionType, dtype: int64

In [23]:
df['victimIP'].value_counts()

162.244.30.100           447932
172.31.13.124            216754
172.31.14.66             154869
::                       111054
255.255.255.255           29418
::ffff:162.244.30.100     26893
0                          7199
                             23
Name: victimIP, dtype: int64

In [50]:
len(jsons)

994692

In [51]:
chan_count = {}
for j in jsons:
    chan = j['channel']
    if chan not in chan_count:
        chan_count[chan] = 0
    else:
        chan_count[chan] += 1
chan_count

{'glastopf.events': 7198,
 'amun.events': 371316,
 'snort.alerts': 40445,
 'dionaea.connections': 575179,
 'dionaea.capture': 542,
 'mwbinary.dionaea.sensorunique': 6}

In [28]:
# used for showing format of various honeypot returns
# d_ct = 0
# for j in jsons:
#     if j['channel'] == 'dionaea.connections':
#         d_ct += 1
#         print(j)
#     if d_ct > 20:
#         break

How many unique attacker IPs do we have?

In [30]:
len(df['attackerIP'].unique())

185936

Ok that's more than expected. 

In [31]:
185936/15000

12.395733333333334

If we were to use https://freegeoip.app/ to get the IP info, it would take us about 13 hours to finish, since the hourly rate limit is 15K.

How much coverage does GeoIP give us?

## GeoIP

In [37]:
import geoip2

ModuleNotFoundError: No module named 'geoip2'

Realized geoip isn't free any more. Will probably use freegeoip.app

# Reputation

I downloaded AlienVault's reputation database at 20191110, 11:01:00 PST (updated hourly).

In [62]:
import numpy as np

In [56]:
colnames = ['IP', 'Reliability', 'Risk', 'Type', 'Country', 'Locale', 'Coords', 'x']
rep = pd.read_csv('../data/reputation_20191110_110100.dat', sep='#', header=None, names=colnames)
def get_lat_long(coords):
    coords = coords.split(',')
    return coords[0], coords[1]
rep["Latitude"], rep["Longitude"] = rep["Coords"].str.split(',', 1).str

In [57]:
rep.head()

,IP,Reliability,Risk,Type,Country,Locale,Coords,x,Latitude,Longitude
0,23.94.213.6,4,2,Malicious Host,US,Los Angeles,"34.0544013977,-118.244003296",3,34.0544013977,-118.244003296
1,23.116.185.114,4,2,Malicious Host,US,NaN,"37.7509994507,-97.8219985962",3,37.7509994507,-97.8219985962
2,14.58.252.223,4,2,Malicious Host,KR,NaN,"37.5111999512,126.974098206",3,37.5111999512,126.974098206
3,45.79.106.170,4,2,Malicious Host,US,Fremont,"37.5483016968,-121.988601685",3,37.5483016968,-121.988601685
4,107.152.104.110,4,2,Malicious Host,US,Jersey City,"40.73279953,-74.0755004883",3,40.73279953,-74.0755004883


In [70]:
rep['Type'].value_counts()

Malicious Host    74892
Name: Type, dtype: int64

In [60]:
rep["Latitude"].head()

0    34.0544013977
1    37.7509994507
2    37.5111999512
3    37.5483016968
4      40.73279953
Name: Latitude, dtype: object

How many IPs are in the big df?

In [61]:
hp_unq_ips = df['attackerIP'].unique()

In [64]:
ip_in_rep = np.isin(hp_unq_ips, rep['IP'])

In [67]:
ips_in_rep = hp_unq_ips[ip_in_rep]

In [69]:
len(ips_in_rep)

417

In [78]:
honeypot_sample = df.sample(n=10000, random_state=1111)

In [79]:
honeypot_sample.shape

(10000, 21)

In [80]:
honeypot_sample['channel'].value_counts()

dionaea.connections    5732
amun.events            3789
snort.alerts            409
glastopf.events          70
Name: channel, dtype: int64

In [81]:
honeypot_sample.to_csv('../data/honeypot_10ksamp.csv', index=False)

In [82]:
honeypot_samp_attackerips = honeypot_sample['attackerIP']

In [83]:
honeypot_samp_attackerips.to_csv('../data/honeypot_10ksamp_attackerips.csv', index=False)

In [92]:
honeypot_all_attackerips = df['attackerIP']

In [93]:
honeypot_all_attackerips = pd.Series(honeypot_all_attackerips.unique())
honeypot_all_attackerips.shape

(185936,)

In [94]:
honeypot_all_attackerips.to_csv('../data/honeypot_all_attackerips.csv', index=False)

## Formulating a Plan

Here's what I plan to do.

Use 3 main data sets:
* Honeypot data. 
* Reputation Data
* freegeoip data, downloaded 15,000 records at a time.

Then, to test, we need
* 10,000k-row sample of honeypot data. 
* Beginnings of a freegeoip data file 

Going to put everything in a Redshift database. Makes the most sense for now given the data size. 

Want the following code:
* Class to set up Redshift database
* Create tables code
    * Staging tables: honeypot, reputation, freegeoip. 
    * Dimension tables: 1 for each of the honeypots, reputation "key" information, geoip "key" information. Time table?  
    * Fact table: Honeypot "touches", with IP, geolocation info, and a few other TBD columns. 
   
* Process to clean honeypot data from json, send to redshift
* Class to download new IPs from freegeoip (sleep 5 mins (300 seconds) between 403s).

In [77]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")

DEBUG:root:test


None
